# 装饰器decorator
> 装饰器可以极大地简化代码，避免每个函数编写重复性代码

Python的decorator本质上就是一个高阶函数，他接受一个函数作为参数，然后返回一个新函数,使用decorator用Python提供的@ 语法，就可以避免写f = decorator(f)这样的语句

In [31]:
def log(f):
    def fn(x) :
        print('call ' + f.__name__ + '()...')
        return f(x)
    return fn

@log
def length(a):
    return len(a)
# 上面的@log 就相当于 f = log(f)
print(length("wo shi test str"))
# call length()...
# 15
print(length)
#<function log.<locals>.fn at 0x000002596FF95C80>


call length()...
15
<function log.<locals>.fn at 0x000002597012C8C8>


+ 解释

从上面可以看出：
 
1. 上面的参数x虽然没有显示的从log函数中传递，python的装饰器其实就是从新返回了一个和我们要装饰的函数有相同参数的新的函数，同时添加了新的功能，又在新函数的内部调用了要装饰的函数，这样就给要装饰的函数添加了新的功能。
 
 2.在调用的时候只需要在函数上面添加**@装饰器名**就可以了。这个就相当于f = log(f)
 
 3.从第二点可以得出一个结论，那就是在使用装饰器后我们的函数已经不是我们之前的函数了。从最后一行的代码我们可以看得出来，它的函数已经变成了log里面的函数fn
 
 4.观察上面的例子我们可以看得出来这个装饰器其实只能接收一个参数的函数，那么如果有好几个参数该怎么办？下面讨论这种情况

## 让装饰器适应任何函数
可以使用Python的\*args和\**kw，保证任意个参数总是能正常调用
理解了上面的第一个例子之后。下面例子就很好理解了，所以就不解释了

In [32]:
def log(f) :
    def fn(*args, **kw):
        print('call ' + f.__name__ + "()...")
        return f(*args, **kw)
    return fn

## 编写带参数的decorator
上面定义的log函数打印的log是定义好的，函数名()...,如果想要自定义log语句，可以为decorator定义参数

In [15]:
def log(prefix):
    def log_decorator(f):
        def wrapper(*args, **kw):
            print("[{0}]{1}()....".format(prefix, f.__name__))
            return f(*args, **kw)
        return wrapper
    return log_decorator

@log('DEBUG')
def test():
    print('test')

print(test())
# [DEBUG]test()....
# test
# None
print(test)
#<function log.<locals>.log_decorator.<locals>.wrapper at 0x000002597012C9D8>


[DEBUG]test()....
test
None
<function log.<locals>.log_decorator.<locals>.wrapper at 0x000002596FFB39D8>


上面我们已经说过了，当我们使用装饰器的时候其实是‘偷梁换柱’把函数给替换了，这样我们在打印函数的__name__的时候函数名称其实就已经改变了
因此我们要把原函数的一些属性复制到新函数中，我们可以使用Python内置的functools可以用来自动化这种“复制任务”

下面我们来看一下我们的装饰器的**最终版本**

In [30]:
# 导入复制任务的包
import functools

def log(prefix):
    def log_decorator(f):
        # 复制函数信息
        @functools.wraps(f)
        def wrapper(*args, **kw):
            print("[{0}]{1}()....".format(prefix, f.__name__))
            return f(*args, **kw)
        return wrapper
    return log_decorator
# 测试

@log("DEBUG")
def test():
    print('I Love py')

test()
print(test.__name__)
print(test.__doc__)


[DEBUG]test()....
I Love py
test
None


从上面打印的信息我们可以看到，函数的信息我们已经替换过来了，这样我们的装饰器的最终版本也就完成了